In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv(r'final_cleaned_preproecessed_file.csv')

In [3]:
overview = data.loc[:, ['id','overview']]
data = data.drop(['id', 'overview', 'IMDB Score','revenue', 'vote_average', 'vote_count','production_company_2','popularity',], axis = 1)

In [4]:
data[data.budget == 0].shape

(4637, 42)

In [5]:
data[data.runtime == 0].shape

(131, 42)

In [6]:
data[data.Lead_Actor == '0'].shape

(521, 42)

In [7]:
data[data.Supporting_Actor == '0'].shape

(392, 42)

In [8]:
data[data.Director == '0'].shape

(56, 42)

In [9]:
data[data.Producer == '0'].shape

(2506, 42)

### Imputing values for Numerical Variable - (runtime )

In [10]:
imputer_runtime = SimpleImputer(missing_values=0, strategy = 'mean')

data.loc[:,['runtime']] = imputer_runtime.fit_transform(data.loc[:,['runtime']])

### Imputing values for Categorical Variables - 
'production_company_1', 'production_company_2', 'Lead_Actor', 'Supporting_Actor', 'Director',
       'Producer'

In [11]:
imputer_categorical = SimpleImputer(missing_values='0', strategy='most_frequent')
cols = ['production_company_1', 'Lead_Actor', 'Supporting_Actor', 'Director','Producer']
data.loc[:,cols] =imputer_categorical.fit_transform(data.loc[:,cols])

### Categorical encoding - Target Encoding

In [12]:
import category_encoders as ce
from category_encoders import TargetEncoder

encoder_lead_actor = ce.TargetEncoder(cols= 'Lead_Actor') 
data.loc[:,'Lead_Actor'] = encoder_lead_actor.fit_transform(data['Lead_Actor'],data['rating'])

encoder_sup_actor = ce.TargetEncoder(cols= 'Supporting_Actor') 
data.loc[:,'Supporting_Actor'] = encoder_sup_actor.fit_transform(data['Supporting_Actor'],data['rating'])

encoder_director_name = ce.TargetEncoder(cols= 'Director') 
data.loc[:,'Director'] = encoder_director_name.fit_transform(data['Director'],data['rating'])

encoder_producer_name = ce.TargetEncoder(cols= 'Producer') 
data.loc[:,'Producer'] = encoder_producer_name.fit_transform(data['Producer'],data['rating'])

encoder_production_company_1 = ce.TargetEncoder(cols= 'production_company_1') 
data.loc[:,'production_company_1'] = encoder_production_company_1.fit_transform(data['production_company_1'],data['rating'])

### Imputing values for budget

In [13]:
train = data[data['budget'] != 0].copy()
test = data[data['budget'] == 0].copy()
X_train, X_test, y_train, y_test =  train.drop('budget',axis = 1), test.drop('budget',axis = 1), train.loc[:,'budget'], test.loc[:,'budget']
ind = y_test.index

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

for i in range(ind.shape[0]):
    data.loc[ind[i],'budget'] = y_pred[i]
    

In [14]:
data.head()

,belongs_to_collection,budget,runtime,num_faces,brightness,saturation,hue,brightness_sd,saturation_sd,hue_sd,...,TV Movie,Thriller,War,Western,production_company_1,Lead_Actor,Supporting_Actor,Director,Producer,rating
0,1,30000000.0,81.0,0.0,184.775439,190.399400,82.805137,58.496526,80.933495,46.984021,...,0.0,0.0,0.0,0.0,3.135069,3.221894,3.218344,3.269310,3.133749,3.598930
1,0,65000000.0,104.0,0.0,166.807096,91.547633,65.296239,83.413251,67.319422,61.296055,...,0.0,0.0,0.0,0.0,3.273118,3.195112,3.133749,3.290745,3.456065,3.760163
2,0,60000000.0,170.0,3.0,45.715307,136.999480,76.802894,60.436514,105.250820,50.140706,...,0.0,1.0,0.0,0.0,3.232329,3.431047,3.433421,3.370673,3.495914,3.905544
3,1,58000000.0,130.0,0.0,109.660769,137.422763,72.647441,86.758819,82.646283,53.146266,...,0.0,1.0,0.0,0.0,3.233043,3.482260,3.013112,2.764595,3.267760,2.740334
4,0,98000000.0,119.0,0.0,163.764743,175.083098,21.076989,78.702461,76.101948,28.865624,...,0.0,0.0,0.0,0.0,3.233850,3.741384,3.451753,3.552290,3.189284,3.710181


### Bag of Words Model with CountVectorizer

In [15]:
"""import re
import nltk
import pickle
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer
def preprocess(df, lst) : 
    for i in range(df.shape[0]) :
        comment = re.sub(r'[?|!|\'|"|#]',r'', df['overview'][i])
        comment = re.sub(r'[.|,|)|(|\|/]',r' ', comment)
        comment = comment.replace("\n"," ")
        comment = re.sub('[^a-zA-Z]',' ', comment)
        comment = comment.lower()
        comment = comment.split()
        ps = SnowballStemmer(language='english')
        comment = [ps.stem(word) for word in comment if not word in set(stopwords.words('english'))]
        comment = ' '.join(comment)
        lst.append(comment)
        if i%1000 == 0 :
            print(str(i)+' comments preprocessed')

    
start1 = datetime.now()    
corpus = []
preprocess(overview, corpus)
end1 = datetime.now()
print('\n..............Corpus created.............')
print('\nTIme taken to create corpus : ', start1 - end1)

cv = CountVectorizer(max_features=500)

ovrvw = cv.fit_transform(corpus).toarray()

pd.DataFrame(ovrvw).to_csv('overview_cv.csv',index_label = False)
"""

ovrvw = pd.read_csv(r'overview_cv.csv')

In [16]:
y = data.loc[:,'rating'].values
X = data.loc[:,:'rating'].values
X.shape

(6634, 42)

In [17]:
X = np.hstack((X,ovrvw))

In [18]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X = sc.fit_transform(X)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from datetime import datetime

start = datetime.now()

"""param = {   "n_estimators"      : [30,50,100],
            "max_features"      : ["auto", "sqrt", "log2"],
            "max_depth"         : [5, 10, 15, 20],
            "bootstrap"         : [True, False],
            "criterion"         : ['mae']
        }"""

#grids = GridSearchCV(RandomForestRegressor(), param_grid= param, cv = 5, scoring = 'neg_mean_absolute_error')
grids = RandomForestRegressor()
grids.fit(X_train,y_train)

end = datetime.now()

print('-------------| Time to fit the model : {} |-------------'.format(start - end))

# print('-------------| Best Paramters : {} |-------------'.format(grids.best_params_))

# print('-------------| Best Score : {} |-------------'.format(-1*grids.best_score_))

C:\Users\ANP\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


-------------| Time to fit the model : -1 day, 23:59:48.655346 |-------------


In [21]:
y_train_pred = grids.predict(X_train)

from sklearn.metrics import mean_absolute_error

print('-------------| Mean Absolute Error on Train set : {} |-------------'.format(mean_absolute_error(y_train, y_train_pred)))

-------------| Mean Absolute Error on Train set : 0.00039754846716733987 |-------------


In [22]:
y_pred = grids.predict(X_test)

from sklearn.metrics import mean_absolute_error

print('-------------| Mean Absolute Error on Test set : {} |-------------'.format(mean_absolute_error(y_test, y_pred)))

-------------| Mean Absolute Error on Test set : 0.0009678417562146343 |-------------
